In [1]:
import requests

In [2]:
URL = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=donnees-synop-essentielles-omm&q=&sort=date&facet=date&facet=nom&facet=nom_epci&facet=nom_dept&facet=nom_reg&facet=tc&rows=10000"
list_meteo = requests.get(URL)
data_collected_meteo = list_meteo.json()

In [3]:
list_annee=[2016,2017,2018,2019,2020,2021]

list_data_collected_meteo_final = []
for i in list_annee:
    filtre = "&refine.date="+str(i)#pour selectionner seulement l'année
    for j in range(1,13):
        list_temperature_annee_mois = requests.get(URL+filtre+"%2F"+str(j)) #pour sélectionner seulement le mois ,1 pour janvier et 12 pour decembre
        list_data_collected_meteo_final.append(list_temperature_annee_mois.json())



In [4]:
len(list_data_collected_meteo_final)

72

In [5]:
#Creation de la base de donnée
from sqlalchemy import create_engine, MetaData, \
    Column, Integer, Numeric, String, Date, Table, ForeignKey, select

engine = create_engine(
    'sqlite:///temperature.sqlite3'
)
engine


Engine(sqlite:///temperature.sqlite3)

In [6]:
metadata = MetaData()

In [7]:
import sqlalchemy
sqlalchemy.__version__ 

'1.4.45'

In [8]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()
Base

sqlalchemy.orm.decl_api.Base

In [9]:
class Temperature(Base):
     __tablename__ = "Temperature"
     record_id = Column(String, primary_key=True)
     date = Column(String)
     nom = Column(String)     
     nom_epci = Column(String) 
     département = Column(String)
     nom_reg = Column(String)
     temperature_celsius = Column(String)

     def __repr__(self):
         return "<Temperature(record_id='%s', date='%s',nom='%s', nom_epci='%s',nom_dept='%s', nom_reg='%s', temperature_celsius='%s')>" % (
             self.record_id,
             self.date,
             self.nom,
             self.nom_epci,
             self.nom_dept,
             self.nom_reg,
             self.temperature_celsius,
         )
    


In [10]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
Session
Base.metadata.create_all(engine)

In [11]:
def generateTemperature():
    """
    Fonction qui stocke les données de la temperature dans la BDD
    """
    Session = sessionmaker(bind=engine)
    session = Session()
    for j in list_data_collected_meteo_final:
        for i in j["records"]:
            try:
                nom_epci_test =i['fields']['nom_epci']
            except KeyError:
                nom_epci_test = "None"
            try:
                nom_dept_test =i['fields']['nom_dept']
            except KeyError:
                nom_epci_test = "None"
            try:
                nom_reg_test =i['fields']['nom_reg']
            except KeyError:
                nom_reg_test = "None"
            try:
                temperature_test =i['fields']['tc']
            except KeyError:
                temperature_test = "None"

            temperature_remplir = Temperature(record_id=i['recordid'], date=i['fields']['date'],
            nom=i['fields']['nom'],
            nom_epci = nom_epci_test ,
            département=nom_dept_test.upper(),    
            nom_reg=nom_reg_test, 
            temperature_celsius=temperature_test)
            session.add(temperature_remplir)
            
    session.commit()
    session.close()


In [12]:
generateTemperature()